<a href="https://colab.research.google.com/github/sarkar-sayan/Feedback-Analysis-using_Machine-Learning-_Sentiment-Analysis_GPT/blob/main/POC_2_Survey_Feedback_Analysis_workingnote.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00


In [3]:
from groq import Groq
!export GROQ_API_KEY='YOUR_GROQCLOUD_API_KEY'

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
import re
import nltk
import os
#from langdetect import detect
#from googletrans import Translator
# Initialize Google Translator
#translator = Translator()

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [8]:
df = pd.read_csv('/content/drive/MyDrive/Sayan RP files/Datasets/Client Feedback Form 1.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 17 columns):
 #   Column                                                                      Non-Null Count  Dtype  
---  ------                                                                      --------------  -----  
 0   Timestamp = A12 + A4                                                        7 non-null      object 
 1   How effectively did the employee communicate with you?                      7 non-null      float64
 2   How promptly did the employee respond to your inquiries or concerns?        7 non-null      float64
 3   How quickly did the employee response your messages ?                       7 non-null      float64
 4   How clear and understandable was the information provided by the employee?  7 non-null      float64
 5   Overall feedback on communication                                           7 non-null      object 
 6   How effectively did the employee understand your bus

In [9]:
# Drop the Timestamp column
df.drop('Timestamp = A12 + A4', axis=1, inplace=True)
#df = df.drop(df.columns[0], axis=1)
df = df.dropna()
df.tail(3)

,How effectively did the employee communicate with you?,How promptly did the employee respond to your inquiries or concerns?,How quickly did the employee response your messages ?,How clear and understandable was the information provided by the employee?,Overall feedback on communication,How effectively did the employee understand your business requirements?,How effectively did the employee handle any issues or concerns you had?,Overall feedback,How knowledgeable was the employee about the technology used?,How knowledgeable was the employee about the Business domain?,Overall feedback.1,How professional was the employee during your interaction?,How courteous and respectful was the employee?,Overall feedback.2,Final Feedback,Final Label
4,3.0,3.0,3.0,3.0,"Overall good, could be better.",4.0,2.0,Employee understood the requirements but could...,4.0,2.0,She is knowledgeable about technology but she ...,2.0,2.0,My problem is not solved still,So-so,negative
5,3.0,4.0,2.0,1.0,Employee was fast and prompt but could not sol...,4.0,2.0,Employee understood the requirements but could...,4.0,2.0,She is knowledgeable about technology but she ...,2.0,3.0,Good in only technology,Problem solved!,positive
6,2.0,1.0,2.0,4.0,I understood the solution but the solution del...,1.0,1.0,Very very bad,1.0,1.0,Total fool,1.0,1.0,Worst service i ever had,I am leaving this service. I will be filing co...,negative


In [ ]:
# Loading and basic preprocessing of dataframe
#df = pd.read_csv('/content/drive/MyDrive/Sayan RP files/Datasets/Client Feedback Form.csv')
#df.drop('Timestamp = A12 + A4', axis=1, inplace=True)
#df = df.drop(df.columns[0], axis=1)
#df = df.dropna()

In [10]:
# Combine all overall feedback columns into one text column
feedback_columns = [col for col in df.columns if 'Overall feedback' in col]
df['All_feedback'] = df[feedback_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df.head(3)

,How effectively did the employee communicate with you?,How promptly did the employee respond to your inquiries or concerns?,How quickly did the employee response your messages ?,How clear and understandable was the information provided by the employee?,Overall feedback on communication,How effectively did the employee understand your business requirements?,How effectively did the employee handle any issues or concerns you had?,Overall feedback,How knowledgeable was the employee about the technology used?,How knowledgeable was the employee about the Business domain?,Overall feedback.1,How professional was the employee during your interaction?,How courteous and respectful was the employee?,Overall feedback.2,Final Feedback,Final Label,All_feedback
0,5.0,1.0,4.0,3.0,this is test feedback,2.0,1.0,good,1.0,5.0,bad,2.0,5.0,good,quite good enough,positive,this is test feedback good bad good
1,5.0,4.0,3.0,2.0,good,4.0,4.0,bad,5.0,2.0,good,4.0,5.0,good,"good, but could be better",positive,good bad good good
2,4.0,4.0,5.0,5.0,"Yes, I am satisfied with the employee who hear...",3.0,5.0,I am pleased with the employee as everything w...,3.0,3.0,Good but there is room for improvement,1.0,4.0,Employee was frank and unprofessional but cour...,My problem has been smoothly solved. I love th...,positive,"Yes, I am satisfied with the employee who hear..."


In [11]:
df.drop(feedback_columns, axis=1, inplace=True)
df.drop('Final Feedback', axis=1, inplace=True)
df.tail(3)

,How effectively did the employee communicate with you?,How promptly did the employee respond to your inquiries or concerns?,How quickly did the employee response your messages ?,How clear and understandable was the information provided by the employee?,How effectively did the employee understand your business requirements?,How effectively did the employee handle any issues or concerns you had?,How knowledgeable was the employee about the technology used?,How knowledgeable was the employee about the Business domain?,How professional was the employee during your interaction?,How courteous and respectful was the employee?,Final Label,All_feedback
4,3.0,3.0,3.0,3.0,4.0,2.0,4.0,2.0,2.0,2.0,negative,"Overall good, could be better. Employee unders..."
5,3.0,4.0,2.0,1.0,4.0,2.0,4.0,2.0,2.0,3.0,positive,Employee was fast and prompt but could not sol...
6,2.0,1.0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,negative,I understood the solution but the solution del...


In [ ]:
# Text preprocessing function
# Translate text if not in English
#def translate_text_if_needed(text):
#    try:
#        language = detect(text)
#        if language != 'en':
#            translated = translator.translate(text, dest='en')
#            return translated.text
#        return text
#    except Exception as e:
#        return text

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    filtered_text = ' '.join([lemmatizer.lemmatize(word) for word in tokens if word not in stop_words])
    return filtered_text

# Apply text preprocessing
#df['All_feedback'] = df['All_feedback'].apply(translate_text_if_needed)
df['All_feedback'] = df['All_feedback'].apply(preprocess_text)
df

In [13]:
# Apply text preprocessing
#df['Final Feedback'] = df['Final Feedback'].apply(translate_text_if_needed)
#df['Final Feedback'] = df['Final Feedback'].apply(preprocess_text)
#df

In [14]:
# Split data into features and target
X = df.drop('Final Label', axis=1)
y = df['Final Label']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [18]:
y

0    positive
1    positive
2    positive
3    negative
4    negative
5    positive
6    negative
Name: Final Label, dtype: object

In [38]:
from sklearn.linear_model import LogisticRegression
# Define ColumnTransformer to separate numerical and text data processing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X.select_dtypes(include='number').columns.tolist()),
        ('text', TfidfVectorizer(), 'All_feedback')
    ],
    remainder='drop'
)

# Define the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

    positive       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



In [49]:
def predict_feedback_proba(pipeline, sample):
    # Get the name of the column containing feedback text
    feedback_column = X_test.columns[0]  # Assuming feedback is in the first column

    # Convert sample to a DataFrame with the correct column name
    # Use the .to_frame().T method to transpose the Series and maintain column names
    sample_df = sample.to_frame().T
    y_prob = pipeline.predict_proba(sample_df)
    prob_pos = y_prob[0][1] * 100  # Probability of being positive
    prob_neg = y_prob[0][0] * 100  # Probability of being negative
    return prob_pos, prob_neg

In [50]:
client = Groq(
    # This is the default and can be omitted
    api_key='YOUR_GROQCLOUD_API_KEY'
    #api_key=os.environ.get("GROQ_API_KEY"),
)

# Function to generate detailed feedback using OpenAI's GPT-4
def generate_detailed_feedback(numerical_data, textual_feedback, prob_pos, prob_neg):
    # Convert numerical_data to a string representation
    numerical_data_str = str(numerical_data)

    prompt = f"Using the following numerical ratings and feedback:\n\nNumerical Ratings: {numerical_data_str}\n\nTextual Feedback: {textual_feedback}\n\nProbability of being positive: {prob_pos:.2f}%\n\nProbability of being negative: {prob_neg:.2f}%\n\nPlease provide a very short final feedback according to the client, i.e, either positive or negative. Not both. If negative, then check if it is very negative, i.e above 70% probability. Give output in the form of either Positive, Mostly Positive, Negative or Mostly Negative with the probability percentage. And then Improvements or Matters to focus upon:"
    chat_completion = client.chat.completions.create(
    messages = [
    {
        "role": "system",
        "content": "you are a helpful assistant."
    },
{
        "role": "user",
        "content": prompt, # Use the formatted prompt here
    },
    ],
    model="llama3-8b-8192" # Add the model keyword argument here
    )
    return chat_completion.choices[0].message.content.strip()  # Extract the content from the response


In [51]:
# Example usage
sample = X.iloc[6]
prob_pos, prob_neg = predict_feedback_proba(pipeline, sample)
print(f"Sample:\n{sample}")
print(f"Probability of being positive: {prob_pos:.2f}%")
print(f"Probability of being negative: {prob_neg:.2f}%")

numerical_data = sample.drop('All_feedback').to_dict()
textual_feedback = sample['All_feedback']
prob_pos, prob_neg = predict_feedback_proba(pipeline, sample)
detailed_feedback = generate_detailed_feedback(numerical_data, textual_feedback, prob_pos, prob_neg)
print(detailed_feedback)

Sample:
How effectively did the employee communicate with you?                                                                      2.0
How promptly did the employee respond to your inquiries or concerns?                                                        1.0
How quickly did the employee response your messages ?                                                                       2.0
How clear and understandable was the information provided by the employee?                                                  4.0
How effectively did the employee understand your business requirements?                                                     1.0
How effectively did the employee handle any issues or concerns you had?                                                     1.0
How knowledgeable was the employee about the technology used?                                                               1.0
How knowledgeable was the employee about the Business domain?                                   

In [52]:
# Example usage
sample = X.iloc[2]
prob_pos, prob_neg = predict_feedback_proba(pipeline, sample)
print(f"Sample:\n{sample}")
print(f"Probability of being positive: {prob_pos:.2f}%")
print(f"Probability of being negative: {prob_neg:.2f}%")

numerical_data = sample.drop('All_feedback').to_dict()
textual_feedback = sample['All_feedback']
prob_pos, prob_neg = predict_feedback_proba(pipeline, sample)
detailed_feedback = generate_detailed_feedback(numerical_data, textual_feedback, prob_pos, prob_neg)
print(detailed_feedback)

Sample:
How effectively did the employee communicate with you?                                                                      4.0
How promptly did the employee respond to your inquiries or concerns?                                                        4.0
How quickly did the employee response your messages ?                                                                       5.0
How clear and understandable was the information provided by the employee?                                                  5.0
How effectively did the employee understand your business requirements?                                                     3.0
How effectively did the employee handle any issues or concerns you had?                                                     5.0
How knowledgeable was the employee about the technology used?                                                               3.0
How knowledgeable was the employee about the Business domain?                                   